In [1]:
import numpy as np

### 1. 4D-Array

In [2]:
x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape # (1, 28, 28)
x[1].shape # (1, 28, 28)

(1, 28, 28)

In [4]:
x[0, 0] # 또는 x[0][0]

array([[7.46940127e-01, 3.28790068e-01, 2.57487041e-01, 2.38409380e-01,
        5.20717197e-01, 7.58866679e-01, 8.46048555e-01, 6.00057178e-02,
        9.64756945e-01, 9.92711449e-01, 5.08910513e-01, 4.11484767e-01,
        9.00296703e-01, 7.43380714e-01, 9.15062255e-01, 7.36367507e-01,
        9.08636576e-01, 4.10605538e-01, 3.14248602e-01, 2.27287299e-01,
        6.46640801e-01, 9.44531864e-01, 8.39508834e-01, 4.26151978e-01,
        5.04543828e-01, 4.09342943e-01, 7.72391598e-01, 6.75821356e-03],
       [7.85132486e-01, 3.91741686e-01, 4.41789284e-01, 1.76548161e-01,
        7.79954943e-01, 5.81431413e-02, 8.17162946e-01, 2.03119670e-01,
        4.41309942e-01, 8.10294617e-01, 8.67842129e-01, 6.69985982e-01,
        4.02494815e-01, 9.29259943e-01, 9.16327118e-02, 4.61237862e-01,
        1.83979932e-01, 2.53442418e-01, 8.64621400e-01, 4.56281411e-01,
        1.10042602e-01, 7.76094040e-02, 1.80522672e-01, 3.77032397e-01,
        4.34506324e-01, 2.61286656e-01, 6.88222096e-01, 4.39644

In [5]:
# im2col(input_data, filter_h, filter_w, stride=1, pad=0)

### 3. Convolution

In [6]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) #(데이터 수, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

(9, 75)
(90, 75)


In [7]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1)
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 4. Pooling

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최대값 (2)
        out = np.max(col, axis=1)
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out